```{contents}
```

## Prompt Template

In **LangChain**, a **Prompt Template** is:

> A **structured, reusable, parameterized prompt object** that separates **prompt logic** from **business logic**.

Instead of hard-coding strings, you define **prompt contracts**.

---

### Why Prompt Templates Exist

Without templates:

* Prompts are scattered strings
* Hard to version, test, or reuse
* Easy to break agents/RAG pipelines

With templates:

* Strong input validation
* Reusability
* Composition with chains/agents
* Provider-agnostic prompts

---

### Core Prompt Template Types

LangChain has **three primary prompt abstractions**:

| Template Type         | Used For                          |
| --------------------- | --------------------------------- |
| `PromptTemplate`      | Text-only models (legacy)         |
| `ChatPromptTemplate`  | Chat models (modern, recommended) |
| `MessagesPlaceholder` | Injecting memory / scratchpad     |

---

### `PromptTemplate` (Text Completion – Legacy)

**Input → string**
**Output → formatted string**


In [1]:

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "Explain {topic} in one sentence"
)

formatted = prompt.format(topic="RAG")
print(formatted)


Explain RAG in one sentence




Use only when:

* Model accepts plain text
* No tools, no agents, no chat

❌ Not recommended for new systems

---

### `ChatPromptTemplate` (Primary & Recommended)

**Input → structured messages**
**Output → list of messages**



In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an IT support assistant"),
    ("human", "{question}")
])

messages = prompt.format_messages(
    question="What is LangChain?"
)
messages

[SystemMessage(content='You are an IT support assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is LangChain?', additional_kwargs={}, response_metadata={})]



Why this matters:

* Preserves role semantics
* Enables tool calling
* Works with agents & RAG

---

### Prompt Templates + LLM (LCEL)

Prompt templates are **Runnable objects**.



In [8]:
from langchain_openai import ChatOpenAI
chain = prompt | ChatOpenAI(temperature=0.2)

result = chain.invoke(
    {"question": "Explain RAG"}
)

print(result.content)


RAG stands for Red, Amber, Green. It is a color-coding system used in project management and reporting to indicate the status or health of a particular task, project, or key performance indicator (KPI).

- Red typically indicates that a task or project is at risk, behind schedule, or facing significant issues that need immediate attention.
- Amber signifies that there are some concerns or potential risks that may impact the task or project if not addressed promptly.
- Green indicates that everything is on track, within schedule, and progressing as planned.

By using the RAG system, stakeholders can quickly assess the status of various aspects of a project or task and prioritize their attention and resources accordingly.




This composability is **core LangChain design**.

---

### Variables & Validation

Prompt templates **declare required inputs**.



In [9]:

prompt = PromptTemplate(
    input_variables=["severity", "issue"],
    template="Severity: {severity}\nIssue: {issue}"
)




If you miss a variable → error early
This prevents **silent prompt bugs**

---

### Partial Variables (Advanced)

Used for **static instructions**.



In [10]:

prompt = PromptTemplate(
    template="{role}\nQuestion: {question}",
    input_variables=["question"],
    partial_variables={"role": "You are a security expert"}
)




Why useful:

* Environment-specific prompts
* Prompt versioning
* Config-driven instructions

---

### `MessagesPlaceholder` (Critical for Agents)

Used to inject:

* Memory
* Tool history
* Scratchpad



In [11]:

from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])




⚠️ Never hard-code scratchpads manually.

---

###  Prompt Templates in Agents

Agents **require** specific variables.

For OpenAI tools agents:

* `{input}`
* `{agent_scratchpad}`



In [12]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "Use tools if needed"),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}")
])




Mismatch → runtime errors (as you observed earlier)

---

### Prompt Templates in RAG



In [13]:

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer using the provided context only"),
    ("human", "Context:\n{context}\n\nQuestion:\n{question}")
])



LangChain injects retrieved documents into `{context}`.

---

### Prompt Templates vs Raw Strings (Key Difference)

| Aspect           | Raw String | PromptTemplate |
| ---------------- | ---------- | -------------- |
| Validation       | ❌          | ✅              |
| Reusability      | ❌          | ✅              |
| Composable       | ❌          | ✅              |
| Agent-safe       | ❌          | ✅              |
| Production-ready | ❌          | ✅              |

---

### Versioning & Testing Prompts

Best practice:

* Treat prompts as **code**
* Store in files
* Version them
* Test with fixed inputs

PromptTemplate makes this feasible.

---

### Common Mistakes

| Mistake                                 | Result              |
| --------------------------------------- | ------------------- |
| Using `PromptTemplate` with chat models | Tool/agent failures |
| Hard-coding scratchpad                  | Agent crashes       |
| Missing variables                       | Runtime KeyError    |
| Embedding logic in prompt text          | Unmaintainable      |

---

### Interview-Ready Summary

> “Prompt Templates in LangChain are structured, validated, composable prompt definitions that decouple prompt engineering from application logic. `ChatPromptTemplate` is the standard for modern, agentic, and RAG-based systems.”

---

### Rule of Thumb

* **Chatbot / Agent / RAG → `ChatPromptTemplate`**
* **Legacy text model → `PromptTemplate`**
* **Agents → always include `agent_scratchpad`**



| Parameter               | Typical Use         | Provided By   |
| ----------------------- | ------------------- | ------------- |
| `{input}`               | User query          | You           |
| `{agent_scratchpad}`    | Tool history        | AgentExecutor |
| `{chat_history}`        | Conversation memory | Memory        |
| `{context}`             | Retrieved docs      | RAG           |
| `{question}`            | Alias of input      | You           |
| `{tools}`               | Tool schemas        | Agent         |
| `{tool_names}`          | Tool list           | Agent         |
| `{system_prompt}`       | Global rules        | Partial vars  |
| `{format_instructions}` | Output schema       | Output parser |
